In [1]:

import fitz
import re
import unidecode
import pandas as pd
import json
import pickle
from tqdm import tqdm
from PIL import Image
from pytesseract import image_to_data

In [2]:
doc = fitz.open("../data/CS_extrait/CA_300960754_9301_1987B09941_2019_20352.pdf")

In [3]:
import os
os.environ['OMP_THREAD_LIMIT'] = '10'

In [4]:
from time import time
start_time = time()
page_list = []

for page in doc:
    pix = page.get_pixmap(dpi=200)
    mode = "RGBA" if pix.alpha else "RGB"
    image = Image.frombytes(mode, [pix.width, pix.height], pix.samples)
    ocr = image_to_data(
        image, lang="fra", config="--psm 1", output_type="data.frame"
    )
    cleaned_ocr = ocr["text"].dropna()
    if cleaned_ocr.empty:
        page_list.append("vide")
    else:
        page_list.append(cleaned_ocr.str.cat(sep=" "))

ocr_time = time() - start_time
ocr_time

49.01874113082886

In [5]:
page_list

["RCES : BOBIGNY Code greffe : 9301 Documents comptables REGISTRE DU COMMERCE ET DES SOCIETES Le greffier du tribunal de commerce de BOBIGNY atteste l'exactitude des informations transmises ci-après Nature du document : Documents comptables (B-5) Numéro de gestion : 1987 B 09941 Numéro SIREN : 300 960 754   Nom ou dénomination : SAINT GOBAIN RECHERCHE Ce dépôt a été enregistré le 06/11/2020 sous le numéro de dépôt 20352",
 '703€ - 6 NOV. 2620 SAINT-GOBAIN RECHERCHE Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019)',
 "Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019) Aux Associés SAINT-GOBAIN RECHERCHE 39 Quai Lucien Lefranc 93300 AUBERVILLIERS Opinion En exécution de la mission qui nous a été confiée par votre assemblée générale, nous avons effectué l'audit des comptes annuels de la société SAINT-GOBAIN RECHERCHE relatifs à l’exercice clos le 31 décembre 2019, tels qu’ils sont joints au présent r

In [6]:
import threading

def ocr_page_to_dict(page, page_number, page_dict):
    pix = page.get_pixmap(dpi=200)
    mode = "RGBA" if pix.alpha else "RGB"
    image = Image.frombytes(mode, [pix.width, pix.height], pix.samples)
    ocr = image_to_data(
        image, lang="fra", config="--psm 1", output_type="data.frame"
    )
    cleaned_ocr = ocr["text"].dropna()
    if cleaned_ocr.empty:
        page_dict[page_number] = "vide"
    else:
        page_dict[page_number] = cleaned_ocr.str.cat(sep=" ")
    return


In [7]:
start_time = time()

# List of threads
threads = []
page_dict = {}

for page_number, page in enumerate(doc):
    threads.append(
        threading.Thread(group=None, target=ocr_page_to_dict, args=(page, page_number, page_dict))
    )

for t in threads:
    t.start()
for t in threads:
    t.join()

# Computing time
ocr_time = time() - start_time
ocr_time

7.475047588348389

In [8]:
page_list = []
for key, value in sorted(page_dict.items()): # Note the () after items!
    page_list.append(value)
page_list

["RCES : BOBIGNY Code greffe : 9301 Documents comptables REGISTRE DU COMMERCE ET DES SOCIETES Le greffier du tribunal de commerce de BOBIGNY atteste l'exactitude des informations transmises ci-après Nature du document : Documents comptables (B-5) Numéro de gestion : 1987 B 09941 Numéro SIREN : 300 960 754   Nom ou dénomination : SAINT GOBAIN RECHERCHE Ce dépôt a été enregistré le 06/11/2020 sous le numéro de dépôt 20352",
 '703€ - 6 NOV. 2620 SAINT-GOBAIN RECHERCHE Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019)',
 "Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019) Aux Associés SAINT-GOBAIN RECHERCHE 39 Quai Lucien Lefranc 93300 AUBERVILLIERS Opinion En exécution de la mission qui nous a été confiée par votre assemblée générale, nous avons effectué l'audit des comptes annuels de la société SAINT-GOBAIN RECHERCHE relatifs à l’exercice clos le 31 décembre 2019, tels qu’ils sont joints au présent r

In [9]:
import sys
sys.path.append("../src/page_selection/")
from utils import extract_document_content_ocr

start_time = time()
page_list = extract_document_content_ocr(doc)

ocr_time = time() - start_time
ocr_time

7.672940969467163

In [10]:
page_list

["RCES : BOBIGNY Code greffe : 9301 Documents comptables REGISTRE DU COMMERCE ET DES SOCIETES Le greffier du tribunal de commerce de BOBIGNY atteste l'exactitude des informations transmises ci-après Nature du document : Documents comptables (B-5) Numéro de gestion : 1987 B 09941 Numéro SIREN : 300 960 754   Nom ou dénomination : SAINT GOBAIN RECHERCHE Ce dépôt a été enregistré le 06/11/2020 sous le numéro de dépôt 20352",
 '703€ - 6 NOV. 2620 SAINT-GOBAIN RECHERCHE Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019)',
 "Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019) Aux Associés SAINT-GOBAIN RECHERCHE 39 Quai Lucien Lefranc 93300 AUBERVILLIERS Opinion En exécution de la mission qui nous a été confiée par votre assemblée générale, nous avons effectué l'audit des comptes annuels de la société SAINT-GOBAIN RECHERCHE relatifs à l’exercice clos le 31 décembre 2019, tels qu’ils sont joints au présent r

Adding semaphore

In [16]:
page_dict = {}
page_dict

{}

In [17]:
def ocr_page_to_dict(page, page_number, page_dict, sema):
    with sema:
        pix = page.get_pixmap(dpi=200)
        mode = "RGBA" if pix.alpha else "RGB"
        image = Image.frombytes(mode, [pix.width, pix.height], pix.samples)
        ocr = image_to_data(
            image, lang="fra", config="--psm 1", output_type="data.frame"
        )
        cleaned_ocr = ocr["text"].dropna()
        if cleaned_ocr.empty:
            page_dict[page_number] = "vide"
        else:
            page_dict[page_number] = cleaned_ocr.str.cat(sep=" ")
        return

In [18]:
start_time = time()

# Semaphor
maxthreads = 10
sema = threading.BoundedSemaphore(value=maxthreads)

# List of threads
threads = []
page_dict = {}

for page_number, page in enumerate(doc):
    threads.append(
        threading.Thread(group=None, target=ocr_page_to_dict, args=(page, page_number, page_dict, sema))
    )

for t in threads:
    t.start()
for t in threads:
    t.join()

# Computing time
ocr_time = time() - start_time
ocr_time

6.711515426635742

In [19]:
page_dict

{1: '703€ - 6 NOV. 2620 SAINT-GOBAIN RECHERCHE Rapport du commissaire aux comptes sur les comptes annuels (Exercice clos le 31 décembre 2019)',
 0: "RCES : BOBIGNY Code greffe : 9301 Documents comptables REGISTRE DU COMMERCE ET DES SOCIETES Le greffier du tribunal de commerce de BOBIGNY atteste l'exactitude des informations transmises ci-après Nature du document : Documents comptables (B-5) Numéro de gestion : 1987 B 09941 Numéro SIREN : 300 960 754   Nom ou dénomination : SAINT GOBAIN RECHERCHE Ce dépôt a été enregistré le 06/11/2020 sous le numéro de dépôt 20352",
 8: "Société : SG RECHERCHE Date de clôture :12/31/2019 | poe ET LE Ée de Produits exceptionnels sur opérations de gestion 2 Produits exceptionnels sur opérations en capital 26 675 Compte de résultat (suite)   PE ER       Reprises sur provisions et transferts de charges Charges exceptionnelles sur opérations de gestion Charges exceptionnelles sur opérations en capital 3 611 294 Dotations exceptionnelles aux amortissements e